In [1]:
# 引進相關套件
import requests
from io import StringIO
import pandas as pd
import numpy as np

In [2]:
# 資料日期
date1 = '20240913'
# 網址
url= 'http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date={}&type=ALL'.format(date1)
print(url)

http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=20240913&type=ALL


In [3]:
# 送出要求，並取得回應資料
response = requests.get(url)

### 下一行若出現錯誤：
### ERROR：NotebookApp.iopub_data_rate_limit=1000000.0

## 修正
### jupyter notebook --generate-config
### 修改 c:\user\<user name>\.jupyter\jupyter_notebook_config.py
### 取消註解 c.NotebookApp.iopub_data_rate_limit = 1000000
### 後面加2個0

In [4]:
print(response.text)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
clean_data=[]
for row in response.text.split('\n'):
    fields=row.split('",')
    if len(fields) == 17 and row[0] != '=':
        clean_data.append(row.replace(' ',''))

csv_data = "\n".join(clean_data)
print(csv_data)

"證券代號","證券名稱","成交股數","成交筆數","成交金額","開盤價","最高價","最低價","收盤價","漲跌(+/-)","漲跌價差","最後揭示買價","最後揭示買量","最後揭示賣價","最後揭示賣量","本益比",
"1101","台泥","10,086,881","3,654","320,138,176","31.50","31.95","31.50","31.80","+","0.30","31.80","165","31.85","705","33.83",
"1101B","台泥乙特","6,218","10","290,732","46.75","46.80","46.75","46.80","","0.00","46.70","1","46.80","8","0.00",
"1102","亞泥","5,487,013","2,759","244,984,207","44.90","44.90","43.90","44.90","+","0.20","44.85","20","44.90","16","14.21",
"1103","嘉泥","107,855","170","1,894,051","17.30","17.70","17.30","17.70","+","0.20","17.65","1","17.70","25","118.00",
"1104","環泥","423,853","330","12,688,140","29.45","30.30","29.45","29.80","+","0.20","29.80","20","29.90","3","9.80",
"1108","幸福","118,166","81","1,727,181","14.55","14.75","14.55","14.60","","0.00","14.60","5","14.65","5","11.77",
"1109","信大","236,348","151","4,323,160","18.05","18.45","18.05","18.45","+","0.35","18.35","5","18.45","22","9.71",
"1110","東泥","359,896","280","7,993,766","21.80","22.4

In [6]:
df = pd.read_csv(StringIO(csv_data))
df.head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
0,1101,台泥,"10,086,881","3,654","320,138,176",31.50,31.95,31.50,31.80,+,0.3,31.80,165,31.85,705,33.83,NaN
1,1101B,台泥乙特,"6,218",10,"290,732",46.75,46.80,46.75,46.80,NaN,0.0,46.70,1,46.80,8,0.00,NaN
2,1102,亞泥,"5,487,013","2,759","244,984,207",44.90,44.90,43.90,44.90,+,0.2,44.85,20,44.90,16,14.21,NaN
3,1103,嘉泥,"107,855",170,"1,894,051",17.30,17.70,17.30,17.70,+,0.2,17.65,1,17.70,25,118.00,NaN
4,1104,環泥,"423,853",330,"12,688,140",29.45,30.30,29.45,29.80,+,0.2,29.80,20,29.90,3,9.80,NaN


In [7]:
# 將以下欄位轉為數值
numeric_columns=['成交股數','成交筆數','成交金額','開盤價','最高價','最低價','收盤價', '本益比']
for i in numeric_columns:
    df[i]=df[i].map(lambda x:x.replace(',', '').replace('--', ''))
    df[i]=pd.to_numeric(df[i])


# 計算漲跌價差
df["漲跌價差"] = df.apply(lambda r: 0-r["漲跌價差"] if r["漲跌(+/-)"] =='-' else r["漲跌價差"], axis=1)

# 計算漲跌幅
df["漲跌幅"] = round(df["漲跌價差"] / (df["收盤價"] - df["漲跌價差"]), 2)  

In [8]:
df

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16,漲跌幅
0,1101,台泥,10086881,3654,320138176,31.50,31.95,31.50,31.80,+,0.30,31.80,165,31.85,705,33.83,NaN,0.01
1,1101B,台泥乙特,6218,10,290732,46.75,46.80,46.75,46.80,NaN,0.00,46.70,1,46.80,8,0.00,NaN,0.00
2,1102,亞泥,5487013,2759,244984207,44.90,44.90,43.90,44.90,+,0.20,44.85,20,44.90,16,14.21,NaN,0.00
3,1103,嘉泥,107855,170,1894051,17.30,17.70,17.30,17.70,+,0.20,17.65,1,17.70,25,118.00,NaN,0.01
4,1104,環泥,423853,330,12688140,29.45,30.30,29.45,29.80,+,0.20,29.80,20,29.90,3,9.80,NaN,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,9944,新麗,90559,123,1827086,20.20,20.20,20.10,20.10,NaN,0.00,20.10,5,20.15,1,59.12,NaN,0.00
1052,9945,潤泰新,4045845,3144,189870958,46.50,47.25,46.45,46.95,+,0.45,46.95,22,47.00,10,9.66,NaN,0.01
1053,9946,三發地產,585152,365,18786238,31.80,32.40,31.45,32.20,+,0.75,32.20,12,32.30,3,43.51,NaN,0.02
1054,9955,佳龍,6521220,3745,240029550,37.20,37.45,36.20,37.00,+,1.75,36.95,11,37.00,26,0.00,NaN,0.05


In [9]:
df = df.drop(df.columns[-2], axis=1)
df.head()
# df.to_csv('每日收盤行情.csv', index=False)

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,漲跌幅
0,1101,台泥,10086881,3654,320138176,31.50,31.95,31.50,31.8,+,0.3,31.80,165,31.85,705,33.83,0.01
1,1101B,台泥乙特,6218,10,290732,46.75,46.80,46.75,46.8,NaN,0.0,46.70,1,46.80,8,0.00,0.00
2,1102,亞泥,5487013,2759,244984207,44.90,44.90,43.90,44.9,+,0.2,44.85,20,44.90,16,14.21,0.00
3,1103,嘉泥,107855,170,1894051,17.30,17.70,17.30,17.7,+,0.2,17.65,1,17.70,25,118.00,0.01
4,1104,環泥,423853,330,12688140,29.45,30.30,29.45,29.8,+,0.2,29.80,20,29.90,3,9.80,0.01
